In [1]:
import tensorflow as tf

In [2]:
rnn_sigle = tf.nn.rnn_cell.BasicRNNCell(200)
#构造一个序列， 长度6 batch 5, 特征100
x = tf.random_normal([6,5,100])
init_state = rnn_sigle.zero_state(5, tf.float32)
state = init_state

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [4]:
outputs= []
for i in range(6):
    if i>0:
        tf.get_variable_scope().reuse_variables()
    out, state = rnn_sigle(x[i], state)
    outputs.append(out)

In [22]:
init_state.shape

TensorShape([Dimension(5), Dimension(200)])

In [6]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
print(state.shape)

print(len(outputs))

(5, 200)


In [23]:
#dynamic_rnn
out, final_state = tf.nn.dynamic_rnn(rnn_sigle, x, initial_state=init_state, time_major=True)
print(final_state.shape)
print(sess.run(final_state)[:,:6])

(5, 200)
[[-0.04559175 -0.37242442 -0.655521    0.7766603   0.521263   -0.80455273]
 [ 0.46494737  0.97858495  0.5433805  -0.13542993  0.12379953 -0.63065827]
 [-0.40336892 -0.4017918  -0.04248516 -0.12526512 -0.48862267  0.9345258 ]
 [-0.29342023 -0.18346852 -0.07702596  0.26332256  0.9613456   0.09616972]
 [ 0.3814289   0.8751279  -0.54198205 -0.08251179  0.08179384  0.05573547]]


In [24]:
print(out.shape)

(6, 5, 200)


In [30]:
#定义初始状态，由随机正太分布组成
init_state = tf.random_normal([5,200], dtype=tf.float32)
out, final_state = tf.nn.dynamic_rnn(rnn_sigle, x, initial_state=init_state, time_major=True)
print(sess.run(final_state[:, :6]))

[[-0.6832235   0.919807   -0.50370914  0.4006028  -0.58549273 -0.9435618 ]
 [-0.76398146  0.8777282   0.5475998  -0.35521835  0.24853514 -0.9175199 ]
 [ 0.627411   -0.23815066  0.6143807   0.71693516 -0.41454247 -0.47755322]
 [-0.06619327 -0.11835966  0.4526294  -0.2660058  -0.09451395 -0.8761625 ]
 [-0.5311825  -0.96382225  0.91070575  0.5084235   0.58590436  0.20521036]]


In [28]:
#RNN训练的过程容易产生过拟合
def build_rnn(num_units, batch_size, keep_prop = 1):
    cell = tf.nn.rnn_cell.BasicRNNCell(num_units)
    cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prop)
    return cell
dropout_cell = build_rnn(100, 3, 0.5)

# LSTM

In [31]:
#lstm中我们可以定义深层次网络，    LSTM 和 MultiRNNcell
def build_lstm(num_units, num_layers, batch_size, keep_prop =1):
    def build_cell(num_units):
        cell = tf.nn.rnn_cell.LSTMCell(num_units, reuse=tf.AUTO_REUSE)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prop)
        return cell
    cell = tf.nn.rnn_cell.MultiRNNCell([build_cell(num_units) for _ in range(num_layers)])
    init_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, init_state

lstm_cell, lstm_init_state = build_lstm(100, 2, 5)
lstm, final_state = tf.nn.dynamic_rnn(lstm_cell, x, initial_state=lstm_init_state, time_major=True)

In [36]:
for i, layer_state in enumerate(final_state):
    print('layer {}'.format(i))
    print('c.shape: {}'.format(layer_state.c.shape))
    print('h.shape: {}'.format(layer_state.h.shape))
    

print(lstm.shape)

layer 0
c.shape: (5, 100)
h.shape: (5, 100)
layer 1
c.shape: (5, 100)
h.shape: (5, 100)


### 自定义状态初始化

不用零去初始化，考虑到lstm的特殊性， tensorflow 用LSTMStatusTuple表示一个LSTMCell的状态，参数如下：

状态c, 状态h

In [38]:
init_state = tuple([tf.nn.rnn_cell.LSTMStateTuple(tf.random_normal([5,100]),
                                                  tf.random_normal([5,100])) for _ in range(2)])

init_state

for i, layer_sate in enumerate(init_state):
    print('layer: {}'.format(i))
    print('c.shape: {}'.format(layer_state.c.shape))
    print('h.shape: {}'.format(layer_state.h.shape))

lstm, final_sate = tf.nn.dynamic_rnn(lstm_cell, x, initial_state=init_state, time_major=True)

## GRU

In [43]:
def build_gru(num_units, num_layers, batch_size, keep_prob=1):
    def build_cell(num_units):
        cell = tf.nn.rnn_cell.GRUCell(num_units, reuse=tf.AUTO_REUSE)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prob)
        return cell
    cell = tf.nn.rnn_cell.MultiRNNCell([build_cell(num_units) for _ in range(num_layers)])
    init_state = cell.zero_state(batch_size, tf.float32)
    return cell, init_state

gru_cell, gru_init_state = build_gru(100, 2, 5)
print(gru_init_state)
gru, final_state = tf.nn.dynamic_rnn(gru_cell, x, initial_state=gru_init_state, time_major=True)